In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
scraped = pd.read_pickle('data/scraped/scraped_with_decs.pickle.gz', compression='gzip')
scraped.head(3)

,name,image_path,price,url,raw_html,URL_name,description
0,Dom Perignon Lenny Kravitz Limited Edition wit...,/product/images/fl_progressive/lmgmud1xsenlouw...,199.97,/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Dom-Perignon-Lenny-Kravitz-Limited-Ed...,Dom Pérignon Vintage 2008 has been distinguish...
1,Louis Roederer Cristal Brut with Two Flutes an...,/product/images/fl_progressive/iqec7e0nf6mzyof...,329.99,/product/Louis-Roederer-Cristal-Brut-with-Two-...,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Louis-Roederer-Cristal-Brut-with-Two-...,Gift pack containing one bottle of 2008 Louis ...
2,Laurent-Perrier Cuvee Rose,/product/images/fl_progressive/10521.jpg,79.99,/product/Laurent-Perrier-Cuvee-Rose/10521,"<div class=""prodItem_wrap"">\n<div class=""prodI...",/product/Laurent-Perrier-Cuvee-Rose/10521,The Cuvée Rosé from Laurent-Perrier is the mos...


In [4]:
name_desc_txt = scraped[['name','description']]

In [9]:
name_desc_txt['start_token'] = "<|startoftext|>"
name_desc_txt['end_token'] = "<|endoftext|>"
name_desc_txt['prompt_token'] = "[prompt]"
name_desc_txt['response_token'] = "[response]"

name_desc_txt = name_desc_txt[
  [
    'start_token', 
    'prompt_token', 
    'name', 
    'response_token', 
    'description',
    'end_token'
  ]
]

In [10]:
name_desc_txt.head()

,start_token,prompt_token,name,response_token,description,end_token
0,<|startoftext|>,[prompt],Dom Perignon Lenny Kravitz Limited Edition wit...,[response],Dom Pérignon Vintage 2008 has been distinguish...,<|endoftext|>
1,<|startoftext|>,[prompt],Louis Roederer Cristal Brut with Two Flutes an...,[response],Gift pack containing one bottle of 2008 Louis ...,<|endoftext|>
2,<|startoftext|>,[prompt],Laurent-Perrier Cuvee Rose,[response],The Cuvée Rosé from Laurent-Perrier is the mos...,<|endoftext|>
3,<|startoftext|>,[prompt],Piper-Heidsieck Cuvee Brut in Travel Case with...,[response],#95 Wine Spectator Top 100 of 2019The Brut rep...,<|endoftext|>
4,<|startoftext|>,[prompt],Clarendon Hills Astralis Syrah 2011,[response],"The pride of our portfolio, Clarendon Hills As...",<|endoftext|>


# Quality cleanup

In [22]:
print(name_desc_txt.shape)
name_desc_clean = name_desc_txt[~name_desc_txt['name'].str.lower().str.contains('gift')]
print(name_desc_txt_clean.shape)
name_desc_clean = name_desc_clean[~name_desc_clean['description'].str.lower().str.contains('gift')]
print(name_desc_clean.shape)


(125787, 6)
(125482, 6)
(125482, 6)


In [23]:
name_desc_clean.head()

,start_token,prompt_token,name,response_token,description,end_token
2,<|startoftext|>,[prompt],Laurent-Perrier Cuvee Rose,[response],The Cuvée Rosé from Laurent-Perrier is the mos...,<|endoftext|>
3,<|startoftext|>,[prompt],Piper-Heidsieck Cuvee Brut in Travel Case with...,[response],#95 Wine Spectator Top 100 of 2019The Brut rep...,<|endoftext|>
4,<|startoftext|>,[prompt],Clarendon Hills Astralis Syrah 2011,[response],"The pride of our portfolio, Clarendon Hills As...",<|endoftext|>
5,<|startoftext|>,[prompt],Yalumba Patchwork Shiraz 2014,[response],This wine shows all the hallmarks of Barossa S...,<|endoftext|>
7,<|startoftext|>,[prompt],Caymus Napa Valley Cabernet Sauvignon 2018,[response],Caymus has a signature style that is dark in c...,<|endoftext|>


# Data split and save

In [24]:
train, test = train_test_split(name_desc_clean, test_size=0.2, random_state=0)
print(train.shape)
print(test.shape)

(100385, 6)
(25097, 6)


In [26]:
name_desc_clean.to_csv(
    "data/scraped/name_desc_nlp_ready.txt", 
    sep='\t',
    index=False,
    header=None)

train.to_csv(
    "data/scraped/name_desc_nlp_ready_train.txt", 
    sep='\t',
    index=False,
    header=None)

test.to_csv(
    "data/scraped/name_desc_nlp_ready_test.txt", 
    sep='\t',
    index=False,
    header=None)